# Make Resnet

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import time
from sklearn.metrics import accuracy_score, recall_score

### 1. Data preprocessing

In [21]:
BATCH_SIZE = 100
LAYER_NM = 18
path_data = './data'

if not os.path.exists(path_data):
    os.mkdir(path_data)

transform_ = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
# transform_ = transforms.Compose([
#     transforms.CenterCrop(256),
#     transforms.RandomCrop(224, 224), # crop to 227x227 size
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor()
# ])
train_data = datasets.STL10(root=path_data, split='train', download=True, transform=transform_)
test_data = datasets.STL10(root=path_data, split='test', download=True, transform=transform_)

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_data,batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)

# define classes
classes = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [22]:
print(next(iter(train_data))[0].size())
print(next(iter(train_loader))[0].size())

torch.Size([3, 128, 128])
torch.Size([100, 3, 128, 128])


### 2. Build Model

In [23]:
class BasicBlockLower(nn.Module):
    def __init__(self, i, in_planes, planes, dropout, stride=1, debugging=False):
        super(BasicBlockLower, self).__init__()
        self.debugging = debugging
        
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                            stride=stride, padding=1, bias=False)        
        self.bn1 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(dropout)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [24]:
class BasicBlockUpper(nn.Module):
    def __init__(self, i, in_planes, planes, dropout, stride=1, debugging=False):
        super(BasicBlockUpper, self).__init__()
        self.debugging = debugging
        self.mul = 4
        self.in_planes = in_planes
        self.planes = planes
        self.stride = stride
        if i == 0 and stride == 1:
            dim = in_planes
        elif i == 0 and stride != 1:
            dim = planes * 2
        else:
            dim = in_planes * self.mul
        self.conv1 = nn.Conv2d(dim, planes, kernel_size=1,
                            stride=stride, padding=0, bias=False)        
        self.bn1 = nn.BatchNorm2d(planes)
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dropout2 = nn.Dropout(dropout)

        self.conv3 = nn.Conv2d(planes, planes * self.mul, kernel_size=1,
                               stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.mul)

        self.shortcut = nn.Sequential()
        if stride != 1 or dim != planes * self.mul:
            self.shortcut = nn.Sequential(
                nn.Conv2d(dim, planes * self.mul,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.mul)
            )

    def forward(self, x):
        if self.debugging == True:
            print('block start\n x size:', x.size(), 'in_planes:', self.in_planes, 'planes:', self.planes, 'stride:', self.stride)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout1(out)
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.dropout2(out)
        out = self.bn3(self.conv3(out))
        if self.debugging == True:
            print('before shortcut \n x size:', x.size(), 'out size:', out.size())
        residual = self.shortcut(x)
        if self.debugging == True:
            print('before add \n out size:', out.size(), 'redisual size:', residual.size())
        out += residual
        out = F.relu(out)
        return out

In [25]:
class ResNet(nn.Module):
    def __init__(self, num_blocks, name, dropout=0.2, debugging=False, num_classes=10):
        super(ResNet, self).__init__()
        self.name = name
        self.debugging = debugging
        self.in_planes = 64
        self.dropout = dropout
        if self.name =='resnet18' or self.name == 'resnet34':
            self.linear_dim = 2048
        else:
            self.linear_dim = 512

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.linear = nn.Linear(self.linear_dim, num_classes)
        self.flatten = nn.Flatten()

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for i, stride in enumerate(strides):
            if self.name == 'resnet18' or self.name == 'resnet34':
                layers.append(BasicBlockLower(i, self.in_planes, planes, self.dropout, stride, self.debugging))
            else:
                layers.append(BasicBlockUpper(i, self.in_planes, planes, self.dropout, stride, self.debugging))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        if self.debugging is True:
            print('x:', x.size())
        out = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), kernel_size=3, stride=2, padding=1)
        if self.debugging is True:
            print('start:', out.size())
        out = self.layer1(out)
        if self.debugging is True:
            print('layer1 out:', out.size())
        out = self.layer2(out)
        if self.debugging is True:
            print('layer2 out:', out.size())
        out = self.layer3(out)
        if self.debugging is True:
            print('layer3 out:', out.size())
        out = self.layer4(out)
        if self.debugging is True:
            print('layer4 out:', out.size())
        out = F.avg_pool2d(out, 2)
        if self.debugging is True:
            print('avg_pool out:', out.size())
        # out = out.view(out.size(0), -1)
        out = self.flatten(out)
        if self.debugging is True:
            print('falten out:', out.size())
        out = self.linear(out)
        return out

### 3. Train

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 1e-2
model_d = next(iter(train_loader))[0].size()[-2] * next(iter(train_loader))[0].size()[-1]
block_list = {'resnet18':[2,2,2,2], 'resnet34':[3,4,6,3], 'resnet50':[3,4,6,3], 
              'resnet101':[3,4,23,3], 'resnet152':[3,8,36,3]}
y_dim = 10 # output class
name = 'resnet18'
dropout = 0.2
model = ResNet(block_list[name], name, dropout, debugging=False).to(device)
# model = ResNet(model_d, y_dim).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

df_loss = pd.DataFrame(columns=['loss'])
epochs = 300
batch_size = len(train_loader)
for i in range(epochs):
    model.train()
    running_loss = 0.
    for image, label in train_loader:
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    df_loss.loc[i, 'loss'] = round(running_loss/batch_size, 4)
    print(f'epoch: {i}, Loss:{running_loss/batch_size:.7f}, lr: {(optimizer.param_groups[0]["lr"]):.7f}')
    scheduler.step()

epoch: 0, Loss:3.6165166, lr: 0.0100000
epoch: 1, Loss:2.2735507, lr: 0.0100000
epoch: 2, Loss:1.9700285, lr: 0.0100000
epoch: 3, Loss:1.8186704, lr: 0.0100000
epoch: 4, Loss:1.6583126, lr: 0.0100000
epoch: 5, Loss:1.5543356, lr: 0.0100000
epoch: 6, Loss:1.5155859, lr: 0.0100000
epoch: 7, Loss:1.4332354, lr: 0.0100000
epoch: 8, Loss:1.3614715, lr: 0.0100000
epoch: 9, Loss:1.3196906, lr: 0.0100000
epoch: 10, Loss:1.2376663, lr: 0.0100000
epoch: 11, Loss:1.1782168, lr: 0.0100000
epoch: 12, Loss:1.0977924, lr: 0.0100000
epoch: 13, Loss:1.0362404, lr: 0.0100000
epoch: 14, Loss:0.9399720, lr: 0.0100000
epoch: 15, Loss:0.8406241, lr: 0.0100000
epoch: 16, Loss:0.7698904, lr: 0.0100000


KeyboardInterrupt: 

In [ ]:
df_loss

,loss
0,7.1009
1,2.6836
2,2.6042
3,2.288
4,1.9422
...,...
295,0.0009
296,0.0008
297,0.0011
298,0.0009


In [ ]:
correct = 0
total = 0
test_result = pd.DataFrame(columns=['output', 'y'])
with torch.no_grad():
    model.eval()
    for i, (input, label) in enumerate(test_loader):
        x = input.to(device)
        y = label.to(device)
        outputs = model(x)
        # value, pred = torch.max(outputs, 1)
        pred = torch.argmax(outputs, 1)
        out = pd.DataFrame(np.array([pred.detach().cpu().numpy(), y.detach().cpu().numpy()]).T, columns=['output', 'y'])
        # print(out)
        test_result = pd.concat([test_result, out], axis=0).reset_index(drop=True)

        total += y.size(0)
        correct += (torch.argmax(outputs, 1) == y).sum().item()
        
    print("test accuracy : {}%".format((100 * correct / total)))

test accuracy : 63.3875%


In [ ]:
test_result['correct'] = (test_result['output'] == test_result['y']) * 1.0
test_result

,output,y,correct
0,8,8,1.0
1,2,2,1.0
2,3,4,0.0
3,7,7,1.0
4,9,9,1.0
...,...,...,...
7995,1,4,0.0
7996,9,9,1.0
7997,0,0,1.0
7998,0,8,0.0


In [ ]:
test_result.correct.value_counts()

correct
1.0    5071
0.0    2929
Name: count, dtype: int64

In [ ]:
acc = accuracy_score(list(test_result['y'].values), list(test_result['output'].values))
rcll = recall_score(list(test_result['y'].values), list(test_result['output'].values), average='micro')
print('Accuracy:', acc, '  Recall:', rcll)

Accuracy: 0.633875   Recall: 0.633875
